In [15]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
from nltk.corpus.reader import ConllCorpusReader
from nltk.util import LazyMap, LazyConcatenation


In [26]:
class newConllReader(ConllCorpusReader):

    def iob_words(self, fileids=None, tagset=None, column="chunk"):
        """
        :return: a list of word/tag/IOB tuples
        :rtype: list(tuple)
        :param fileids: the list of fileids that make up this corpus
        :type fileids: None or str or list
        """
        self._require(self.WORDS, self.POS, self.CHUNK)
        def get_iob_words(grid):
            return self._get_iob_words(grid, tagset, column)
        return LazyConcatenation(LazyMap(get_iob_words, self._grids(fileids)))

    def _get_iob_words(self, grid, tagset=None, column="chunk"):
        pos_tags = self._get_column(grid, self._colmap['pos'])
        if tagset and tagset != self._tagset:
            pos_tags = [map_tag(self._tagset, tagset, t) for t in pos_tags]
        return list(zip(self._get_column(grid, self._colmap['words']),
                   self._get_column(grid, self._colmap[column])))



In [41]:
train = newConllReader('../../ner_conll2003', 'eng.train', ['words', 'pos','chunk','ne']).iob_words(column='ne')
test = newConllReader('../../ner_conll2003', 'eng.testa', ['words', 'pos','chunk','ne']).iob_words(column='ne')
X_train, Y_train = [x[0] for x in train], [x[1] for x in train]
X_test, Y_test = [x[0] for x in train], [x[1] for x in train]
TAGS = list(set(Y_train)) 
TAGS2i = {TAGS[i]:i for i in range(len(TAGS))}
words = list(set(X_train))